In [1]:
import pandas as pd
from pandas_schema import pandas_schema

In [ ]:
file_path = '/Users/matthewtryba/Desktop/subsampled_data.csv'

loans = pd.read_csv(file_path=file_path, sep='|', header=None, low_memory=False, dtype=pandas_schema)